In [1]:
#setup:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

from wetterdienst.provider.dwd.observation import DwdObservationRequest, DwdObservationDataset, DwdObservationPeriod, DwdObservationResolution
# from wetterdienst import Wetterdienst, Resolution, Period, Parameter
from pprint import pprint

In [2]:
from wetterdienst import Settings
Settings.tidy = True
Settings.humanize =True
Settings.si_units = True
# API = Wetterdienst(provider="dwd", network="observation")

sample_request = DwdObservationRequest(
        parameter=["wind_speed"],
        resolution=DwdObservationResolution.HOURLY,
        start_date="2021-01-01 00:00:00+00:00",
        end_date="2021-01-31 00:00:00+00:00",
        period=DwdObservationPeriod.HISTORICAL,
        )

# print(dir(sample_request))
pprint(sample_request.discover())

def dwd_request(params, start, end, station_ids):
    """Retrieve stations_result of DWD that measure temperature."""
    request = DwdObservationRequest(
        parameter=[params],
        resolution=DwdObservationResolution.HOURLY,
        start_date=start,
        end_date=end,
        period=DwdObservationPeriod.HISTORICAL,
    )
    
    stations = request.filter_by_station_id(station_ids)
    return stations#["station_id"]


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]


{'annual': {'cloud_cover_total': {'origin': '1/8', 'si': 'pct'},
            'count_weather_type_dew': {'origin': '-', 'si': '-'},
            'count_weather_type_fog': {'origin': '-', 'si': '-'},
            'count_weather_type_glaze': {'origin': '-', 'si': '-'},
            'count_weather_type_hail': {'origin': '-', 'si': '-'},
            'count_weather_type_sleet': {'origin': '-', 'si': '-'},
            'count_weather_type_storm_stormier_wind': {'origin': '-',
                                                       'si': '-'},
            'count_weather_type_storm_strong_wind': {'origin': '-', 'si': '-'},
            'count_weather_type_thunder': {'origin': '-', 'si': '-'},
            'precipitation_height': {'origin': 'mm', 'si': 'kg / m ** 2'},
            'precipitation_height_max': {'origin': 'mm', 'si': 'kg / m ** 2'},
            'snow_depth': {'origin': 'cm', 'si': 'm'},
            'snow_depth_new': {'origin': 'cm', 'si': 'm'},
            'sunshine_duration': {'origin': '

In [12]:
selected_stations=pd.read_pickle("../data/Stations_filtered_ids/selected_stations.pkl")
print(selected_stations.head())
selected_station_ids=selected_stations.Stations_id.tolist()
print(selected_station_ids)

   Stations_id  von_datum  bis_datum  Stationshoehe  geoBreite  geoLaenge  \
0          183   19730101   20220909             42    54.6791    13.4344   
1          701   19490101   20220915              7    53.5332     8.5761   
2          856   19910101   20220915            551    47.8843    12.5404   
3          953   19550101   20220915            481    49.7619     7.0542   
4          963   19940103   20220915             38    52.5881     8.3424   

  Stationsname              Bundesland mosmix_id  
0       Arkona  Mecklenburg-Vorpommern     10091  
1  Bremerhaven                  Bremen     10129  
2     Chieming                  Bayern     10982  
3   Deuselbach         Rheinland-Pfalz     10615  
4     Diepholz           Niedersachsen     10321  
[183, 701, 856, 953, 963, 1443, 1605, 1694, 2115, 3231, 4104, 4177]


In [13]:
start_date="2021-01-01 00:00:00+00:00"
end_date="2021-12-31 23:59:00+00:00"
parameters = ["wind_speed", "sunshine_duration", "pressure_air_site", "temperature_air_mean_200", "cloud_cover_total"]
station_ids=[str(i).zfill(5) for i in selected_station_ids]
print(station_ids)
date=pd.date_range(start_date, end_date, freq="H")

['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


header = pd.MultiIndex.from_product([station_ids, parameters], names=['station_id', 'parameter'])
total_df=pd.DataFrame(columns=header)

total_df.loc[:,("date", "date")]=date
total_df.loc[:,("date", "timestamp")]=date.astype('int64')//10**9

for parameter in parameters:
    # station = dwd_request(parameter, start_date, end_date, station_ids)
    for id in station_ids:
        print("Receiving: ", parameter, " from station ", id)
        station = dwd_request(parameter, start_date, end_date, id)
        df = station.values.all().df
        # print(df.value.head())
        # total_df.loc[:, (id, parameter)]=df.value.copy()
        data_df[id]=df.value.copy()

#total_df.head(30)

In [14]:
def get_weatherdata(date, parameter, start_date, end_date, station_ids):
    data_df=pd.DataFrame(columns=station_ids)
    data_df["date"]=date
    data_df["timestamp"]=date.astype('int64')//10**9
    print("Receiving: ", parameter, " from stations: ", station_ids)
    for id in station_ids:
        station = dwd_request(parameter, start_date, end_date, [id])
        df = station.values.all().df
        # print(df.head())
        # print(df.tail())
        data_df[id]=df.value
    return data_df


In [15]:
parameter=parameters[0]
data_df = get_weatherdata(date, parameter, start_date, end_date, station_ids)
data_df.fillna(0)
path="../data/weather/"+parameter+"_2021.pkl"
print(path)
data_df.to_pickle(path)

start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]


Receiving:  wind_speed  from stations:  ['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filter

../data/weather/wind_speed_2021.pkl


In [19]:
for parameter in parameters:
    data_df = get_weatherdata(date, parameter, start_date, end_date, station_ids)
    data_df.fillna(0)
    path="../data/weather/"+parameter+"_2021.pkl"
    print(path)
    data_df.to_pickle(path)


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]


Receiving:  wind_speed  from stations:  ['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filter

../data/weather/wind_speed_2021.pkl
Receiving:  sunshine_duration  from stations:  ['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filter

...

../data/weather/pressure_air_site_2021.pkl
Receiving:  temperature_air_mean_200  from stations:  ['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filter

../data/weather/temperature_air_mean_200_2021.pkl
Receiving:  cloud_cover_total  from stations:  ['00183', '00701', '00856', '00953', '00963', '01443', '01605', '01694', '02115', '03231', '04104', '04177']


start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filtering limited to defined periods [<Period.HISTORICAL: 'historical'>]
start_date and end_date filter

../data/weather/cloud_cover_total_2021.pkl


In [33]:
parameter=parameters[2]
print(parameter)
path="../data/weather/"+parameter+"_2021.pkl"
new_df=pd.read_pickle(path)
new_df

pressure_air_site


,00183,00701,00856,00953,00963,01443,01605,01694,02115,03231,04104,04177,date,timestamp
0,100020.0,100310.0,94120.0,94650.0,99950.0,97670.0,100050.0,99740.0,100270.0,95050.0,96200.0,99090.0,2021-01-01 00:00:00+00:00,1609459200
1,100020.0,100320.0,94100.0,94660.0,99960.0,97690.0,100060.0,99740.0,100290.0,95050.0,96190.0,99080.0,2021-01-01 01:00:00+00:00,1609462800
2,100020.0,100340.0,94090.0,94720.0,99970.0,97750.0,100080.0,99750.0,100320.0,95080.0,96200.0,99150.0,2021-01-01 02:00:00+00:00,1609466400
3,100030.0,100370.0,94070.0,94770.0,100010.0,97780.0,100080.0,99750.0,100330.0,95090.0,96230.0,99200.0,2021-01-01 03:00:00+00:00,1609470000
4,100010.0,100370.0,94030.0,94800.0,100000.0,97790.0,100080.0,99740.0,100320.0,95100.0,96220.0,99200.0,2021-01-01 04:00:00+00:00,1609473600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,100470.0,101480.0,95830.0,96720.0,101270.0,99740.0,101130.0,100570.0,101350.0,96740.0,97850.0,101110.0,2021-12-31 19:00:00+00:00,1640977200
8756,100570.0,101520.0,95940.0,96800.0,101320.0,99840.0,101210.0,100630.0,101400.0,96850.0,97920.0,101240.0,2021-12-31 20:00:00+00:00,1640980800
8757,100640.0,101600.0,96100.0,96840.0,101390.0,99910.0,101280.0,100660.0,101480.0,96890.0,97990.0,101300.0,2021-12-31 21:00:00+00:00,1640984400
8758,100700.0,101690.0,96100.0,96840.0,101460.0,99960.0,101300.0,100730.0,101560.0,96930.0,98040.0,101310.0,2021-12-31 22:00:00+00:00,1640988000


In [22]:
new_df=new_df.fillna(0)
new_df.to_pickle(path)